In [1]:
# From https://stackoverflow.com/a/5849861
import time
class Timer(object):
    def __init__(self, name=None):
        self.name = name

    def __enter__(self):
        self.tstart = time.time()

    def __exit__(self, type, value, traceback):
        print('Elapsed [{}]: {}s'.format(self.name, time.time() - self.tstart))


In [2]:
def betti(C, verbose = False) :
	
	import itertools
	import numpy as np
	import networkx as nx
	from scipy.sparse import lil_matrix
	
	def DIAGNOSTIC(*params) :
		if verbose :
			print(*params)
	
	#
	# 1. Prepare maximal cliques
	#
	
	# Sort each maximal clique, make sure it's a tuple
	# Also, commit C to memory if necessary
	C = [tuple(sorted(c)) for c in C]
	
	DIAGNOSTIC("Number of maximal cliques: {} ({}M)".format(len(C), round(len(C) / 1e6)))
	
	#
	# 2. Enumerate all simplices
	#
	
	# S[k] will hold all k-simplices
	# S[k][s] is the ID of simplex s
	S = []
	for k in range(0, max(len(s) for s in C)) :
		# Get all (k+1)-cliques, i.e. k-simplices, from max cliques mc
		Sk = set(c for mc in C for c in itertools.combinations(mc, k+1))
		# Check that each simplex is in increasing order
		assert(all((list(s) == sorted(s)) for s in Sk))
		# Assign an ID to each simplex, in lexicographic order
		S.append(dict(zip(sorted(Sk), range(0, len(Sk)))))

		DIAGNOSTIC("Number of {}-simplices: {}".format(k, len(Sk)))
	
	# The cliques are redundant now
	del C
		
	# Euler characteristic
	ec = sum(((-1)**k * len(S[k])) for k in range(0, len(S)))
	
	DIAGNOSTIC("Euler characteristic:", ec)
		
	#
	# 3. Construct the boundary operators
	#
	
	# D[k] is the boundary operator
	#      from the k complex
	#      to the k-1 complex
	D = [None for _ in S]
		
	# D[0] maps to zero by definition
	D[0] = lil_matrix( (1, len(S[0])) )

	# Construct D[1], D[2], ...
	for k in range(1, len(S)) :
		D[k] = lil_matrix( (len(S[k-1]), len(S[k])) )
		SIGN = np.asmatrix([(-1)**i for i in range(0, k+1)]).transpose()

		for (ks, j) in S[k].items() :
			# Indices of all (k-1)-subsimplices s of the k-simplex ks
			I = [S[k-1][s] for s in sorted(itertools.combinations(ks, k))]
			D[k][I, j] = SIGN

	# The simplices are redundant now
	del S
	
	for (k, d) in enumerate(D) :
		DIAGNOSTIC("D[{}] has shape {}".format(k, d.shape))
	
	# Check that D[k-1] * D[k] is zero
	assert(all((0 == np.dot(D[k-1], D[k]).count_nonzero()) for k in range(1, len(D))))
	
    
	#
	# 4. Compute rank and dimker of the boundary operators
	#
	
	# Compute rank using matrix SVD
	rk = [np.linalg.matrix_rank(d.todense()) for d in D] + [0]
	# Compute dimker using rank-nullity theorem
	ns = [(d.shape[1] - rk[n]) for (n, d) in enumerate(D)] + [0]

	# The boundary operators are redundant now
	#del D

	DIAGNOSTIC("rk:", rk)
	DIAGNOSTIC("ns:", ns)

	#
	# 5. Infer the Betti numbers
	#
	
	# Betti numbers
	# B[0] is the number of connected components
	B = [(n - r) for (n, r) in zip(ns[:-1], rk[1:])]

	# Remove trailing zeros
	while B and (B[-1] == 0) : B.pop()

	DIAGNOSTIC("Betti numbers:", B)
	
	# Check: Euler-Poincare formula (see Eqn 16 in [Zomorodian])
	epc = sum(((-1)**k * B[k]) for k in range(0, len(B)))
	if (ec != epc) : print("Warning: ec = {} and epc = {} should be equal".format(ec, epc))

	return (B, D)

In [3]:
# Find the rank of a binary matrix over Z/2Z
# (conceptual implementation)
#
# RA, 2017-11-07 (CC-BY-4.0)
#
# Adapted from
#   https://triangleinequality.wordpress.com/2014/01/23/computing-homology/
#
def binary_rank(M) :
    
    # M-pty matrix?
    if not M.count_nonzero() : return 0

    # Find any nonzero entry, i.e. the pivot
    (p, q) = tuple(a[0] for a in M.nonzero())

    # Indices of entries to flip
    # (Could filter out p and q)
    I = M[:, q].nonzero()[0]
    J = M[p, :].nonzero()[1]

    # Flip those entries
    for i in I :
        for j in J :
            M[i, j] = not M[i, j]

    # Zero out pivot row p / column q
    # (Or delete them from the matrix)
    M[p, :] = 0
    M[:, q] = 0
    
    return 1 + binary_rank(M)

In [7]:
def binary_rank2(M) :
    # M-pty matrix?
    if not M.count_nonzero() : return 0
    
    # Convert to dict of sets
    I2J = dict((i, set(M[i, :].nonzero()[1])) for i in range(0, M.shape[0]))
    J2I = dict((j, set(M[:, j].nonzero()[0])) for j in range(0, M.shape[1]))

    I2J = { k : v for (k, v) in I2J.items() if v }
    J2I = { k : v for (k, v) in J2I.items() if v }
        
    rank = 0
    
    def matrix_from(i2j) :
        import numpy as np
        
        M = np.zeros((1 + max(i2j.keys()),  max(1 + max(J) for J in i2j.values() if J)))
        
        for (i, J) in i2j.items() :
            M[i, list(J)] = 1
        
        return M
    
    while J2I :
        I = next(iter(J2I.values()))
        J = I2J[next(iter(I))]

        for i in I : 
            delta = I2J[i].symmetric_difference(J)
            if delta :
                I2J[i] = delta
            else :
                del I2J[i]

        for j in J : 
            delta = J2I[j].symmetric_difference(I)
            if delta :
                J2I[j] = delta
            else :
                del J2I[j]

        rank += 1
    
    return rank

In [140]:
import networkx as nx
import numpy as np

G = nx.gnp_random_graph(7, 0.9, seed=0)
    
(B, D) = betti(nx.find_cliques(G))
d = D[2]
    
print("Matrix size:", d.shape)
    
rank0 = np.linalg.matrix_rank(d.todense())
rank1 = binary_rank(abs(d).tolil())
rank2 = binary_rank2(abs(d).tolil())

print("Rank:", rank0)

print("Ranks agree:", rank0 == rank1, "/", rank0 == rank2)

Matrix size: (17, 19)
Rank: 11
Ranks agree: True / True


In [141]:
import numpy as np
from numpy.linalg import matrix_rank

M = d.todense()
matrix_rank(np.dot(M, M.transpose()))

11

In [142]:
from scipy.sparse import lil_matrix
from numpy.linalg import matrix_rank
A = d.todense()
print(A)
B = A.transpose()
P = np.zeros( (A.shape[0], B.shape[1]) )
for i in range(0, A.shape[0]) :
    for j in range(0, B.shape[1]) :
        z = 0
        for k in range(A.shape[1]) :
            z += (bool(A[i, k]) and bool(B[k, j]))
        z = z % 2
        P[i, j] = z
print("Trace:", np.matrix.trace(P))
print(matrix_rank(P))
print(binary_rank(lil_matrix(P)))

[[ 1.  1.  1.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
   0.]
 [-1.  0.  0.  0.  1.  1.  1.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
   0.]
 [ 0. -1.  0.  0. -1.  0.  0.  0.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.
   0.]
 [ 0.  0. -1.  0.  0. -1.  0.  0. -1.  1.  0.  0.  0.  0.  0.  0.  0.  0.
   0.]
 [ 0.  0.  0. -1.  0.  0. -1.  0.  0. -1.  1.  0.  0.  0.  0.  0.  0.  0.
   0.]
 [ 0.  0.  0.  0.  0.  0.  0. -1.  0.  0. -1.  0.  0.  0.  0.  0.  0.  0.
   0.]
 [ 1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  1.  1.  1.  0.  0.  0.  0.
   0.]
 [ 0.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0. -1.  0.  0.  1.  0.  0.  0.
   0.]
 [ 0.  0.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0. -1.  0. -1.  1.  0.  0.
   0.]
 [ 0.  0.  0.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0. -1.  0. -1.  0.  0.
   0.]
 [ 0.  0.  0.  0.  1.  0.  0.  0.  0.  0.  0.  1.  0.  0.  0.  0.  1.  0.
   0.]
 [ 0.  0.  0.  0.  0.  1.  0.  0.  0.  0.  0.  0.  1.  0.  0.  0. -1.  1.
   0.]
 [ 0.  0.  0.  0.  0.  0.  1

In [160]:
def proj(V, v) :
    if (V.size == 0) : return False
    
    def binmul(a, b) :
        z = sum((i and j) for (i, j) in zip(a, b))
        return int(z % 2)
    
    p = np.asmatrix([binmul(V[:, j], v) for j in range(V.shape[1])])
    #return sum(binmul(V[i, :].transpose(), p.transpose()) for i in range(V.shape[0]))
    return int(sum(p.transpose()))

from random import sample, choice
A = list(range(0, d.shape[1]))
B = []
rank = 0
while len(A) :
    a = choice(A)
    A.remove(a)
    r = binary_rank(d[:, B + [a]])
    
    
    #print((r > rank), proj(d[:, B].todense(), d[:, a].todense()))
    
    if (r > rank) :
        B.append(a)
        rank = r
        print(len(A), rank)
        continue
print("Done")

True False
18 1
True 1
17 2
True 0
16 3
True 0
15 4
True 1
14 5
True 1
13 6
True 4
12 7
True 2
11 8
True 3
10 9
True 3
9 10
True 3
8 11
False 6
False 5
False 4
False 5
False 5
False 6
False 7
False 4
Done
